In [ ]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import random
import traceback


def get_jobs(keyword, num_jobs, locId, verbose):
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''

    # Initializing the webdriver
    options = webdriver.ChromeOptions()

    # Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    # options.add_argument('headless')

    # Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="chromedriver", options=options)
    driver.set_window_size(1120, 1000)

    # url
    url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=" + \
        keyword+"&sc.keyword="+keyword+"&locT=C&locId="+locId+"&jobType="
    
    #url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []

    time.sleep(1)
    driver.find_element(By.XPATH,
        './/li[@class="react-job-listing css-bkasv9 eigr9kq0"]').click()

    time.sleep(1)
    try:
        # clicking to the X.
        driver.find_element(By.CSS_SELECTOR,'[alt="Close"]').click()
        print(' x out worked')
    except NoSuchElementException:
        print(' x out failed')
        pass

    # If true, should be still looking for new jobs.
    while len(jobs) < num_jobs:
        try:
            stale = False
            # Going through each job in this page
            # jl for Job Listing. These are the buttons we're going to click.
            job_buttons = driver.find_elements(By.XPATH,
                ".//*[contains(@class,'react-job-listing')]")
            print(len(job_buttons))
            for job_button in job_buttons:

                print("Progress: {}".format(
                    "" + str(len(jobs)) + "/" + str(num_jobs)))
                if len(jobs) >= num_jobs:
                    break
                try:
                    job_button.click()  # You might
                    print('clicked')
                except StaleElementReferenceException:
                    print("Stale element found")
                    stale = True
                    break

                sleep_time = random.randint(1,6)
                time.sleep(sleep_time)

                collected_successfully = False

                while not collected_successfully:
                    try:
                        company_name = driver.find_element(By.XPATH,
                            './/div[@class="css-xuk5ye e1tk4kwz5"]').text.split("\n")[0]
                        location = driver.find_element(By.XPATH,
                            './/div[@class="css-56kyx5 e1tk4kwz1"]').text
                        job_title = driver.find_element(By.XPATH,
                            './/div[contains(@class, "css-1j389vi e1tk4kwz2")]').text
                        job_description = driver.find_element(By.XPATH,
                            './/div[@class="jobDescriptionContent desc"]').text
                        collected_successfully = True
                        print("Collected successfully")
                    except Exception as e:
                        print("Exception" + str(e))
                        time.sleep(2)
                        continue

                try:
                    salary_estimate = driver.find_element(By.XPATH,
                        './/span[@class="css-1hbqxax e1wijj240"]').text
                    print("Collected successfully salary")
                except NoSuchElementException:
                    print("No such element exception")
                    salary_estimate = -1  # You need to set a "not found value. It's important."

                try:
                    rating = driver.find_element(By.XPATH,
                        './/span[@class="css-1m5m32b e1tk4kwz4"]').text
                    print("Collected successfully rating")
                except NoSuchElementException:
                    print ("No rating")
                    rating = -1  # You need to set a "not found value. It's important."

                # Printing for debugging
                if verbose:
                    print("Job Title: {}".format(job_title))
                    print("Salary Estimate: {}".format(salary_estimate))
                    print("Job Description: {}".format(job_description[:500]))
                    print("Rating: {}".format(rating))
                    print("Company Name: {}".format(company_name))
                    print("Location: {}".format(location))

                # Going to the Company tab...
                # clicking on this:
                # <div class="tab" data-tab-type="overview"><span>Company</span></div>
                try:
    #                 driver.find_element(By.XPATH,
    #                     './/div[@data-test="overview"]').click()

    #                 time.sleep(2)

                    try:
                        size = driver.find_element(By.XPATH,
                            './/div[@class="d-flex justify-content-start css-daag8o e1pvx6aw2"]//span[text()="Size"]//following-sibling::*').text
                        print("Collected successfully size")
                    except NoSuchElementException:
                        print("No size")
                        size = -1

                    try:
                        founded = driver.find_element(By.XPATH,
                            './/div[@class="d-flex justify-content-start css-daag8o e1pvx6aw2"]//span[text()="Founded"]//following-sibling::*').text
                        print("Collected successfully FOUNDED")
                    except NoSuchElementException:
                        founded = -1

                    try:
                        type_of_ownership = driver.find_element(By.XPATH,
                            './/div[@class="d-flex justify-content-start css-daag8o e1pvx6aw2"]//span[text()="Type"]//following-sibling::*').text
                        print("Collected successfully TYPE")
                    except NoSuchElementException:
                        type_of_ownership = -1

                    try:
                        industry = driver.find_element(By.XPATH,
                            './/div[@class="d-flex justify-content-start css-daag8o e1pvx6aw2"]//span[text()="Industry"]//following-sibling::*').text
                        print("Collected successfully industry")
                    except NoSuchElementException:
                        industry = -1

                    try:
                        sector = driver.find_element(By.XPATH,
                            './/div[@class="d-flex justify-content-start css-daag8o e1pvx6aw2"]//span[text()="Sector"]//following-sibling::*').text
                    except NoSuchElementException:
                        sector = -1

                    try:
                        revenue = driver.find_element(By.XPATH,
                            './/div[@class="d-flex justify-content-start css-daag8o e1pvx6aw2"]//span[text()="Revenue"]//following-sibling::*').text
                    except NoSuchElementException:
                        revenue = -1

                # Rarely, some job postings do not have the "Company" tab.
                except NoSuchElementException:
                    print("No company")
                    size = -1
                    founded = -1
                    type_of_ownership = -1
                    industry = -1
                    sector = -1
                    revenue = -1

                if verbose:
                    print("Size: {}".format(size))
                    print("Founded: {}".format(founded))
                    print("Type of Ownership: {}".format(type_of_ownership))
                    print("Industry: {}".format(industry))
                    print("Sector: {}".format(sector))
                    print("Revenue: {}".format(revenue))
                    print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

                jobs.append({"Job Title": job_title,
                             "Salary Estimate": salary_estimate,
                             "Job Description": job_description,
                             "Rating": rating,
                             "Company Name": company_name,
                             "Location": location,
                             "Size": size,
                             "Founded": founded,
                             "Type of ownership": type_of_ownership,
                             "Industry": industry,
                             "Sector": sector,
                             "Revenue": revenue})
                # add job to jobs

            # Clicking on the "next page" button
            try:
                if not stale:
                    driver.find_element(By.XPATH,
                        './/button[@class="nextButton css-1hq9k8 e13qs2071"]').click()
                    print("Going to the next page")
                else:
                    print("Stale, not going to next page")
            except NoSuchElementException:
                print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(
                    num_jobs, len(jobs)))
                raise
        except Exception as e:
            traceback.print_tb(e.__traceback__)
            print("Had an exception, retrieved {}.".format(len(jobs)))
            time.sleep(10)
            continue
            

    # This line converts the dictionary object into a pandas DataFrame.
    return pd.DataFrame(jobs)

In [ ]:
#San Francisco, CA
# 1147401 is the location ID of SF
df_SF = get_jobs('data scientist', 100,'1147401',False)
df_SF.to_csv('df_SF.csv',index = False)
# to check numvber of duplicate job postings
df_SF.duplicated().sum()

In [ ]:
# to check the column names
df_SF.info()

In [ ]:
# NYC, NY
df_NYC= get_jobs('data scientist', 100, '1132348',False)
df_NYC.to_csv('df_NYC.csv',index = False)
df_NYC.duplicated().sum()

In [ ]:
# Raleigh, NC
df_Raleigh= get_jobs('data scientist', 100, '1138960',False)
df_Raleigh.to_csv('df_Raleigh.csv',index = False)
df_Raleigh.duplicated().sum()

In [ ]:
# Philadelphia, PA
df_Phili= get_jobs('data scientist', 100, '1152672', False)
df_Phili.to_csv('df_Philadelphia.csv',index = False)
df_Phili.duplicated().sum()

In [ ]:
# Dallas, TX
df_Dallas= get_jobs('data scientist', 100,'1139977',False)
df_Dallas.to_csv('df_Dallas.csv',index = False)
df_Dallas.duplicated().sum()

In [ ]:
# Miami, Fl
df_Miami = get_jobs('data scientist', 100,'1154170',False)
df_Miami.to_csv('df_Miami.csv',index = False)
df_Miami.duplicated().sum()

In [ ]:
#Atlanta, Ga
df_Atlanta = get_jobs('data scientist', 100,'1155583',False)
df_Atlanta.to_csv('df_Atlanta.csv',index = False)
df_Atlanta.duplicated().sum()

In [ ]:
#Detroit, MI
df_Detroit = get_jobs('data scientist', 100,'1134644',False)
df_Detroit.to_csv('df_Detroit.csv',index = False)
df_Detroit.duplicated().sum()

In [ ]:
#Cleveland, OH
df_Cleveland = get_jobs('data scientist', 100,'1145778',False)
df_Cleveland.to_csv('df_Cleveland.csv',index = False)
df_Cleveland.duplicated().sum()

In [ ]:
# Denver, CO
df_Denver = get_jobs('data scientist', 100,'1148170',False)
df_Denver.to_csv('df_Denver.csv',index = False)
df_Denver.duplicated().sum()

In [ ]:
# Orlando, FL
df_Orlando = get_jobs('data scientist', 100,'1154247',False)
df_Orlando.to_csv('df_Orlando.csv',index = False)
df_Orlando.duplicated().sum()

In [ ]:
# St.louis, Mo
df_St_Louis = get_jobs('data scientist', 100,'1131270',False)
df_St_Louis.to_csv('df_St_Louis.csv',index = False)
df_St_Louis.duplicated().sum()

In [ ]:
# Charlotte, Nc
df_Charlotte = get_jobs('data scientist', 100,'1138644',False)
df_Charlotte.to_csv('df_Charlotte.csv',index = False)
df_Charlotte.duplicated().sum()

In [ ]:
# Salt Lake City, Ut
df_Salt_Lake_City = get_jobs('data scientist', 100,'1128289',False)
df_Salt_Lake_City.to_csv('df_Salt_Lake_City.csv',index = False)
df_Salt_Lake_City.duplicated().sum()

In [ ]:
# Columbus, Oh
df_Columbus = get_jobs('data scientist', 100,'1145845',False)
df_Columbus.to_csv('df_Columbus.csv',index = False)
df_Columbus.duplicated().sum()

In [ ]:
# Las Vegas, Nv
df_Las_Vegas = get_jobs('data scientist', 100,'1149603',False)
df_Las_Vegas.to_csv('df_Las_Vegas.csv',index = False)
df_Las_Vegas.duplicated().sum()

In [ ]:
# Kansas City, Mo
df_Kansas_City = get_jobs('data scientist', 100,'1131040',False)
df_Kansas_City.to_csv('df_Kansas_City.csv',index = False)
df_Kansas_City.duplicated().sum()

In [ ]:
#Indianapolis, In
df_Indianapolis = get_jobs('data scientist', 100,'1145013',False)
df_Indianapolis.to_csv('df_Indianapolis.csv',index = False)
df_Indianapolis.duplicated().sum()

In [ ]:
# Cincinnati, Oh
df_Cincinnati = get_jobs('data scientist', 100,'1145705',False)
df_Cincinnati.to_csv('df_Cincinnati.csv',index = False)
df_Cincinnati.duplicated().sum()

In [ ]:
# Phoenix, Az
df_Phoenix = get_jobs('data scientist', 100,'1133904',False)
df_Phoenix.to_csv('df_Phoenix.csv',index = False)
df_Phoenix.duplicated().sum()

In [ ]:
# Portland, OR
df_Portland = get_jobs('data scientist', 100,'1151614',False)
df_Portland.to_csv('df_Portland.csv',index = False)
df_Portland.duplicated().sum()

In [ ]:
# Houston, Tx
df_Houston = get_jobs('data scientist', 100,'1140171',False)
df_Houston.to_csv('df_Houston.csv',index = False)
df_Houston.duplicated().sum()

In [ ]:
# Seattle, WA
df_Seattle = get_jobs('data scientist', 100,'1150505',False)
df_Seattle.to_csv('df_Seattle.csv',index = False)
df_Seattle.duplicated().sum()

In [ ]:
# Austin, Tx
df_Austin = get_jobs('data scientist', 100,'1139761',False)
df_Austin.to_csv('df_Austin.csv',index = False)
df_Austin.duplicated().sum()

In [ ]:
# Boston, MA
df_Boston = get_jobs('data scientist', 100,'1154532',False)
df_Boston.to_csv('df_Boston.csv',index = False)
df_Boston.duplicated().sum()

In [ ]:
# Washington, DC
df_Washington_DC = get_jobs('data scientist', 100,'1138213',False)
df_Washington_DC.to_csv('df_Washington_DC.csv',index = False)
df_Washington_DC.duplicated().sum()

In [ ]:
# Arlington, VA
df_Arlington = get_jobs('data scientist', 100,'1130337',False)
df_Arlington.to_csv('df_Arlington.csv',index = False)
df_Arlington.duplicated().sum()

In [ ]:
# Los Angeles, CA
df_LA = get_jobs('data scientist', 100,'1146821',False)
df_LA.to_csv('df_LA.csv',index = False)
df_LA.duplicated().sum()

In [ ]:
# Chicago, IL
df_Chicago = get_jobs('data scientist', 100,'1128808',False)
df_Chicago.to_csv('df_Chicago.csv',index = False)
df_Chicago.duplicated().sum()

In [ ]:
# Minneapolis, MN
df_Minneapolis = get_jobs('data scientist', 100,'1142551',False)
df_Minneapolis.to_csv('df_Minneapolis.csv',index = False)
df_Minneapolis.duplicated().sum()

In [ ]:
# Baltimore, MD
df_Baltimore = get_jobs('data scientist', 100,'1153527',False)
df_Baltimore.to_csv('df_Baltimore.csv',index = False)
df_Baltimore.duplicated().sum()

In [ ]:
# San Diego, CA
df_San_Diego = get_jobs('data scientist', 100,'1147311',False)
df_San_Diego.to_csv('df_San_Diego.csv',index = False)
df_San_Diego.duplicated().sum()

In [ ]:
# Irvine, CA
df_Irvine_CA = get_jobs('data scientist', 100,'1146798',False)
df_Irvine_CA.to_csv('df_Irvine_CA.csv',index = False)
df_Irvine_CA.duplicated().sum()

In [ ]:
# Tampa, FL
df_Tampa = get_jobs('data scientist', 100,'1154429',False)
df_Tampa.to_csv('df_Tampa.csv',index = False)
df_Tampa.duplicated().sum()

In [ ]:
# Nashville, TN
df_Nashville= get_jobs('data scientist', 58,'1144541',False)
df_Nashville.to_csv('df_Nashville.csv',index = False)
df_Nashville.duplicated().sum()

In [ ]:
# Santa Barbara
df_Santa_Barbara= get_jobs('data scientist', 100,'1147473',False)
df_Santa_Barbara.to_csv('df_Santa_Barbara.csv',index = False)
df_Santa_Barbara.duplicated().sum()

In [ ]:
# Oakland, CA
df_Oakland= get_jobs('data scientist', 60,'1147380',False)
df_Oakland.to_csv('df_Oakland.csv',index = False)
df_Oakland.duplicated().sum()

In [ ]:
# Fremont, CA
df_Fremont= get_jobs('data scientist', 100,'1147355',False)
df_Fremont.to_csv('df_Fremont.csv',index = False)
df_Fremont.duplicated().sum()

In [ ]:
# Honolulu, HI 
df_Honolulu= get_jobs('data scientist', 60,'1140656',False)
df_Honolulu.to_csv('df_Honolulu.csv',index = False)
df_Honolulu.duplicated().sum()